In [11]:
from typing import List
import numpy as np
import imageio
import cv2
import copy
import glob
import os
import matplotlib.pyplot as plt


In [12]:
# CALIBRACION

def load_images(filenames: List) -> List:
    return [imageio.imread(filename) for filename in filenames]


def show_image(img, window_name = 'Image'):
    cv2.imshow(window_name, img)
    cv2.waitKey()
    cv2.destroyAllWindows() 


def get_chessboard_points(chessboard_shape, dx, dy):
    points = np.empty([0, 3])
    for i in range(chessboard_shape[1]):
        for j in range(chessboard_shape[0]):
            points = np.vstack((points, np.array([dx*i, dy*j, 0])))
    return points


In [13]:
imgs_path = glob.glob('../calibracion/*.jpg')  
imgs = load_images(imgs_path)
size = (7,6)

C:\Users\laugo\AppData\Local\Temp\ipykernel_33744\1110238016.py:4: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  return [imageio.imread(filename) for filename in filenames]


In [14]:
corners = [cv2.findChessboardCorners((img), size) for img in imgs]

#copying the corners 
corners_copy = copy.deepcopy(corners)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.01)

# To refine corner detections with cv2.cornerSubPix() you need to input grayscale images. Build a list containing grayscale images.
imgs_gray = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in imgs]

corners_refined = [cv2.cornerSubPix(i, cor[1], size, (-1, -1), criteria) if cor[0] else [] for i, cor in zip(imgs_gray, corners_copy)]
imgs_copy = copy.deepcopy(imgs)

#  Use cv2.drawChessboardCorners() to draw the cornes
draw_imgs = []
for i in range(0,len(imgs)):
    draw_imgs.append(cv2.drawChessboardCorners(imgs[i], patternSize=size, corners=corners[i][1], patternWasFound=corners[i][0]))

In [15]:
real_points = get_chessboard_points(size, 30, 30)
# Filter data and get only those with adequate detections
valid_corners = [cor[1] for cor in corners if cor[0]]
# Convert list to numpy array
valid_corners = np.asarray(valid_corners, dtype=np.float32)

In [16]:
object_points = np.asarray([real_points for i in range(len(valid_corners))], dtype=np.float32)
image_points = np.asarray(valid_corners, dtype=np.float32)

#calibrating camera
rms, intrinsics, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(object_points, image_points, (320, 240), None, None)

# Obtain extrinsics
extrinsics = list(map(lambda rvec, tvec: np.hstack((cv2.Rodrigues(rvec)[0], tvec)), rvecs, tvecs))

#saving the calibration file
np.savez('calib_rasp', intrinsic=intrinsics, extrinsic=extrinsics)

In [18]:
print("Intrinsics:")
print(np.array2string(intrinsics, formatter={'all': lambda x: f'{x:6.2f}'}))
print("\nDistortion coefficients:")
print(np.array2string(dist_coeffs, formatter={'all': lambda x: f'{x:6.4f}'}))
print(f"\nRoot mean squared reprojection error: {rms:6.4f}")

Intrinsics:
[[1093.81   0.00 207.91]
 [  0.00 1096.71 248.58]
 [  0.00   0.00   1.00]]

Distortion coefficients:
[[-0.0290 1.4915 -0.0018 0.0010 -16.3624]]

Root mean squared reprojection error: 0.1777
